In [1]:
from alice import Alice
from bob import Bob

def execute_qkd(pairs, DEBUG = True):
    # Initialization
    alice = Alice(pairs)
    bob = Bob()
    
    # Send pairs
    pairs = alice.prepare()

    # Measure each pair and retrieve double matchings
    bob = Bob()
    double_matchings = bob.measure(pairs)
    
    # Verify that there's double matchings
    if not len(double_matchings):
        print("Key exchange failed, there's no double matchings. Please, try again.")
        return None
    
    # Compute usable frames for Bob
    usable_frames, usable_frames_types = alice.compute_usable_frames(double_matchings)
    
    # Veirfy that there's usable frames
    if not len(usable_frames):
        print("Key exchange failed, there's no usable frames. Please, try again.")
        return None

    # Compute sifting string
    sifting_string = bob.compute_sifting_string(usable_frames)

    # Bob computes secret key
    bob_key = bob.generate_shared_key(usable_frames)

    # Alice computes secret key
    alice_key = alice.generate_shared_key(usable_frames_types, sifting_string)
    
    if alice_key == bob_key:
        if DEBUG:
            print("Key exchange completed, here's the data:")
            print(f"Alice send pairs: {alice.pairs_data}")
            print(f"Bob send double matchings: {double_matchings}")
            print(f"Alice send usable frames: {usable_frames}")
            print(f"Bob send sifting string: {sifting_string}")
            print(f"Bob shared key: {bob_key}")
            print(f"Alice shared key: {alice_key}")
        return (usable_frames, sifting_string), (alice_key, bob_key)
    else:
        print("Key exchange failed, both keys are not equal. Please, try again.")
        return None

In [45]:
public, private = execute_qkd(16)

Key exchange completed, here's the data:
Alice send pairs: {0: '0x,1z', 1: '0x,1z', 2: '1x,1z', 3: '0x,0z', 4: '0x,1z', 5: '1x,0z', 6: '0x,1z', 7: '0x,0z', 8: '0x,0z', 9: '1x,1z', 10: '0x,1z', 11: '1x,1z', 12: '1x,1z', 13: '0x,0z', 14: '0x,1z', 15: '0x,0z'}
Bob send double matchings: [0, 1, 2, 3, 4, 5, 6, 9, 10, 12, 14, 15]
Alice send usable frames: [(0, 2), (0, 5), (0, 9), (0, 12), (1, 2), (1, 5), (1, 9), (1, 12), (2, 4), (2, 5), (2, 6), (2, 10), (2, 14), (4, 5), (4, 9), (4, 12), (5, 6), (5, 9), (5, 10), (5, 12), (5, 14), (6, 9), (6, 12), (9, 10), (9, 14), (10, 12), (12, 14)]
Bob send sifting string: ['01', '10', '10', '01', '01', '10', '10', '01', '00', '11', '01', '01', '00', '11', '11', '00', '10', '00', '10', '11', '11', '10', '01', '10', '11', '01', '00']
Bob shared key: 100000101000001001111111011111010000001010001000101001
Alice shared key: 100000101000001001111111011111010000001010001000101001


In [4]:
from partial_key_recovery import attack

public, private = execute_qkd(128, DEBUG = False)
usable_frames, sifting_string = public
alice_key, bob_key = private
assert alice_key == bob_key

key_recovered = attack(usable_frames, sifting_string)

private_key_blocks = [ alice_key[i : i + 2] for i in range(0, len(private[0]), 2)  ]
key_recovered_blocks = [ key_recovered[i : i + 2] for i in range(0, len(key_recovered), 2)  ]

for x, y in zip(private_key_blocks, key_recovered_blocks):
    print(x, y)

11 11
01   
11 11
11 11
01   
01   
01   
11 11
01   
01   
01   
11 11
01   
01   
11 11
01   
01   
01   
11 11
11 11
01   
11 11
01   
01   
01   
01   
01   
01   
01   
11 11
01   
10 10
00   
00   
10 10
10 10
10 10
10 10
10 10
10 10
10 10
10 10
00   
10 10
10 10
00   
10 10
10 10
00   
10 10
10 10
00   
00   
10 10
00   
10 10
10 10
00   
00   
10 10
10 10
00   
10 10
01   
01   
01   
01   
11 11
01   
01   
01   
11 11
11   
01   
11   
01   
11 11
11   
11 11
01   
01   
11   
01   
11   
11   
01   
01   
01   
01   
11 11
11   
01   
01   
10 10
10 10
10 10
10 10
00   
10   
10   
10   
00   
00   
10 10
00   
10 10
00   
00   
00   
10   
10 10
00   
10 10
00   
00   
10   
10   
10   
10   
00   
00   
10   
10 10
10 10
10 10
10 10
10 10
00   
10   
10   
10   
00   
00   
10 10
00   
10 10
00   
00   
00   
10   
10 10
00   
10 10
00   
00   
10   
10   
10   
10   
00   
00   
10   
10 10
01   
01   
01   
11 11
01   
01   
01   
11 11
01   
01   
11 11
01   
01   
01  